# Parse Vplus Browser PR + Package Sheets

In [ ]:
import pandas as pd
from pathlib import Path
# excel_file = Path(r'E:\nextcloud\py_knecht\RenderKnecht\_TestDocs\20171109_MBV_A3-Familie_AoA_MY2018.xlsx')
excel_file = Path(r'I:\Nextcloud\py\py_knecht\RenderKnecht\_TestDocs\20171109_MBV_A3-Familie_AoA_MY2018.xlsx')

1. Parse excel file, skipping first 2 rows, combining row 2+3 as header

In [5]:
ef = pd.ExcelFile(excel_file)

In [6]:
ef.sheet_names

['Modelle', 'Farben 8V', 'PR-Nummern', 'Pakete', 'Parameter']

In [3]:
df = pd.read_excel(excel_file, sheet_name='PR-Nummern', skiprows=[0,1], header=[0,1])
df.head(3)

PR-Familie      PR-FamilienText          PR-Nummer  \
  Unnamed: 0_level_1   Unnamed: 1_level_1 Unnamed: 2_level_1   
0                AAU  Aktionsausführungen                NaN   
1                NaN                  NaN                E0A   
2                NaN                  NaN                E3T   

                      Text             Modell  8VM            
        Unnamed: 3_level_1 Unnamed: 4_level_1  B8L  RWY  S1Y  
0                      NaN                NaN  NaN  NaN  NaN  
1  Keine Aktionsausführung                NaN    L    L    L  
2                   "Life"                NaN    P    -    -

2. Join the header rows containing the model string spread across two rows

In [4]:
#  Join Header Rows

def join_header_rows(rows):
    first_row, second_row = rows
    
    if second_row.startswith('Unnamed'):
        return first_row
    else:
        return first_row + second_row
    
df.columns = df.columns.map(join_header_rows)
df.head(3)

,PR-Familie,PR-FamilienText,PR-Nummer,Text,Modell,8VMB8L,8VMRWY,8VMS1Y
0,AAU,Aktionsausführungen,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,E0A,Keine Aktionsausführung,NaN,L,L,L
2,NaN,NaN,E3T,"""Life""",NaN,P,-,-


3. Clean up empty rows and columns

In [5]:
# Remove empty rows
df.dropna(thresh=2, inplace=True)

# Remove empty descriptive column 'Modell'
df.drop(columns=['Modell'], inplace=True)

df.head(10)

,PR-Familie,PR-FamilienText,PR-Nummer,Text,8VMB8L,8VMRWY,8VMS1Y
0,AAU,Aktionsausführungen,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,E0A,Keine Aktionsausführung,L,L,L
2,NaN,NaN,E3T,"""Life""",P,-,-
4,ABR,Abschließbare Radschrauben,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,1PA,Radschrauben Standard,L,L,L
7,AED,Fahrzeugklassendifferenzierung Aggregate / Pla...,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,7LJ,Fahrzeugklassen-Differenzierung 8V0/8VC,L,L,L
10,AER,Abgasendrohr,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,0P0,Abgasendrohre,L,L,L
12,NaN,NaN,0P6,RS-Sportabgasanlage,-,P,-


In [6]:
# Forward fill PR-Family column and convert to category
df['PR-Familie'] = df['PR-Familie'].ffill().astype('category')

# Forward fill PR-Family Description column and convert to category
df['PR-FamilienText'] = df['PR-FamilienText'].ffill().astype('category')

df.head(10)

,PR-Familie,PR-FamilienText,PR-Nummer,Text,8VMB8L,8VMRWY,8VMS1Y
0,AAU,Aktionsausführungen,NaN,NaN,NaN,NaN,NaN
1,AAU,Aktionsausführungen,E0A,Keine Aktionsausführung,L,L,L
2,AAU,Aktionsausführungen,E3T,"""Life""",P,-,-
4,ABR,Abschließbare Radschrauben,NaN,NaN,NaN,NaN,NaN
5,ABR,Abschließbare Radschrauben,1PA,Radschrauben Standard,L,L,L
7,AED,Fahrzeugklassendifferenzierung Aggregate / Pla...,NaN,NaN,NaN,NaN,NaN
8,AED,Fahrzeugklassendifferenzierung Aggregate / Pla...,7LJ,Fahrzeugklassen-Differenzierung 8V0/8VC,L,L,L
10,AER,Abgasendrohr,NaN,NaN,NaN,NaN,NaN
11,AER,Abgasendrohr,0P0,Abgasendrohre,L,L,L
12,AER,Abgasendrohr,0P6,RS-Sportabgasanlage,-,P,-


In [7]:
# Drop rows without PR
df.dropna(subset=['PR-Nummer'], inplace=True)

df.head(10)

,PR-Familie,PR-FamilienText,PR-Nummer,Text,8VMB8L,8VMRWY,8VMS1Y
1,AAU,Aktionsausführungen,E0A,Keine Aktionsausführung,L,L,L
2,AAU,Aktionsausführungen,E3T,"""Life""",P,-,-
5,ABR,Abschließbare Radschrauben,1PA,Radschrauben Standard,L,L,L
8,AED,Fahrzeugklassendifferenzierung Aggregate / Pla...,7LJ,Fahrzeugklassen-Differenzierung 8V0/8VC,L,L,L
11,AER,Abgasendrohr,0P0,Abgasendrohre,L,L,L
12,AER,Abgasendrohr,0P6,RS-Sportabgasanlage,-,P,-
15,AGM,Abgaskonzept,0GT,"Abgaskonzept, SULEV",L,-,-
16,AGM,Abgaskonzept,7GB,"Abgaskonzept, ULEV 2",-,-,L
17,AGM,Abgaskonzept,7MU,"Abgaskonzept, LEV3 / Tier3 125",-,L,-
20,AHV,Anhängevorrichtung,1D0,Ohne Anhängevorrichtung,L,L,L


4. Example for extracting a trim line

In [9]:
model = df[['PR-Familie', 'PR-FamilienText', 'PR-Nummer', 'Text', '8VMB8L']]
model.set_index(['PR-Familie'], inplace=True)

trim = model[model['8VMB8L'].str.match('L')]
trim.head(10)

,PR-FamilienText,PR-Nummer,Text,8VMB8L
PR-Familie,,,,
AAU,Aktionsausführungen,E0A,Keine Aktionsausführung,L
ABR,Abschließbare Radschrauben,1PA,Radschrauben Standard,L
AED,Fahrzeugklassendifferenzierung Aggregate / Pla...,7LJ,Fahrzeugklassen-Differenzierung 8V0/8VC,L
AER,Abgasendrohr,0P0,Abgasendrohre,L
AGM,Abgaskonzept,0GT,"Abgaskonzept, SULEV",L
AHV,Anhängevorrichtung,1D0,Ohne Anhängevorrichtung,L
AIB,Airbag,4UB,Airbag für NAR,L
AKB,Aktivkohlebehälter / Ottopartikelfilter,1E7,"Aktivkohlebehälter ORVR,für verschärftes Verda...",L
ALG,Ablagenpaket,QE0,Ohne Ablagenpaket/box,L


5. Example for extracting options

In [47]:
options = model[model['8VMB8L'].str.match('E')]
options.head(10)

,PR-FamilienText,PR-Nummer,Text,8VMB8L
PR-Familie,,,,
COC,ABE-Nachträge,C01,Betriebserlaubnis Nachtrag,E
COC,ABE-Nachträge,C02,Betriebserlaubnis Nachtrag,E
GKH,Gewichtsklasse Hinterachse,0YA,Gewichtsbereich 1 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YB,Gewichtsbereich 2 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YC,Gewichtsbereich 3 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YD,Gewichtsbereich 4 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YE,Gewichtsbereich 5 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YF,Gewichtsbereich 6 nur Einbausteuerung keine Be...,E
GKH,Gewichtsklasse Hinterachse,0YG,Gewichtsbereich 7 nur Einbausteuerung keine Be...,E
